In [ ]:
!pip install transformers==4.41.0, datasets==3.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: tr

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import os
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
from datasets import load_dataset
ds = load_dataset("community-datasets/per_sent")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.84M [00:00<?, ?B/s]

test_random-00000-of-00001.parquet:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

test_fixed-00000-of-00001.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3355 [00:00<?, ? examples/s]

Generating test_random split:   0%|          | 0/579 [00:00<?, ? examples/s]

Generating test_fixed split:   0%|          | 0/827 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/578 [00:00<?, ? examples/s]

In [ ]:
class RoBERTa():
  def __init__(self):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.tokenizer = AutoTokenizer.from_pretrained("roberta-base")
    self.model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=4)
    self.model.to(self.device)

  def get_model(self):
    return self.model

  def get_tokenizer(self):
    return self.tokenizer

In [ ]:
#make func to turn text into tokens and then use the hugging face dataset .map with that func to get tokenized dataset
def tokenize_the_text(persent_text):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokenized = tokenizer(
        persent_text["MASKED_DOCUMENT"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # add the labels
    tokenized["labels"] = persent_text["TRUE_SENTIMENT"]
    return tokenized
#didnt make a datasetloader class like in hw3 since this does the same thing
tokenized_ds = ds.map(tokenize_the_text, batched=True)


Map:   0%|          | 0/3355 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/579 [00:00<?, ? examples/s]

Map:   0%|          | 0/827 [00:00<?, ? examples/s]

Map:   0%|          | 0/578 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
ds = load_dataset("community-datasets/per_sent")

In [ ]:
#make func to turn text into tokens and then use the hugging face dataset .map with that func to get tokenized dataset
def tokenize_the_text(persent_text):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokenized = tokenizer(
        persent_text["MASKED_DOCUMENT"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # add the labels
    tokenized["labels"] = persent_text["TRUE_SENTIMENT"]
    return tokenized
#didnt make a datasetloader class like in hw3 since this does the same thing
tokenized_ds = ds.map(tokenize_the_text, batched=True)


In [ ]:
#we need to make the evaluation metrics function ourselves for the hugging face trainer
def eval_metrics(preds_and_labels):
    preds, labels = preds_and_labels #grab the predictions and labels from the tuple argument
    preds = preds.argmax(-1) #grab largest probability
    accuracy = accuracy_score(labels, preds) #avg =weighted for multiclass, ecept for accuracy
    precision = precision_score(labels, preds, zero_division=0,average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average = "weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
train_data = tokenized_ds["train"]
val_data = tokenized_ds["validation"]
test_random_data = tokenized_ds["test_random"]
test_fixed_data = tokenized_ds["test_fixed"]

In [ ]:
roberta_model = RoBERTa()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#make trainer, we're using huggingface Trainer

learning_rate = 3e-5
batch_size = 16
num_epochs = 3 #we start with these but we will change during hyperparam tuning

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model= roberta_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = roberta_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.935145,0.525952,0.276625,0.525952,0.362561
2,No log,0.935023,0.517301,0.440438,0.517301,0.426721


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.935145,0.525952,0.276625,0.525952,0.362561
2,No log,0.935023,0.517301,0.440438,0.517301,0.426721
3,0.959700,0.930278,0.539792,0.478404,0.539792,0.500102


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=630, training_loss=0.9557393270825583, metrics={'train_runtime': 259.6659, 'train_samples_per_second': 38.761, 'train_steps_per_second': 2.426, 'total_flos': 2648260326666240.0, 'train_loss': 0.9557393270825583, 'epoch': 3.0})

In [ ]:
#f1 is .5 at epoch 3 with valdation loss of .93 and training loss of .959

#Hyperparam tuning

In [ ]:
roberta_model = RoBERTa()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

learning_rate = 2e-5 #lower lr
batch_size = 16
num_epochs = 3

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

trainer = Trainer(
    model= roberta_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = roberta_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.937964,0.525952,0.276625,0.525952,0.362561
2,No log,0.933097,0.529412,0.511303,0.529412,0.378952


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.937964,0.525952,0.276625,0.525952,0.362561
2,No log,0.933097,0.529412,0.511303,0.529412,0.378952
3,0.959000,0.930283,0.519031,0.457955,0.519031,0.480247


TrainOutput(global_step=630, training_loss=0.9538641308981275, metrics={'train_runtime': 258.2817, 'train_samples_per_second': 38.969, 'train_steps_per_second': 2.439, 'total_flos': 2648260326666240.0, 'train_loss': 0.9538641308981275, 'epoch': 3.0})

In [ ]:
#same val and train loss but lower f1

In [ ]:
roberta_model = RoBERTa()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

learning_rate = 2e-5 #lower lr
batch_size = 16
num_epochs = 5 #more epoch

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

trainer = Trainer(
    model= roberta_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = roberta_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.933181,0.525952,0.276625,0.525952,0.362561
2,No log,0.940164,0.525952,0.276625,0.525952,0.362561
3,0.961300,0.947175,0.543253,0.508312,0.543253,0.420138
4,0.961300,0.918601,0.550173,0.485731,0.550173,0.507829


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.933181,0.525952,0.276625,0.525952,0.362561
2,No log,0.940164,0.525952,0.276625,0.525952,0.362561
3,0.961300,0.947175,0.543253,0.508312,0.543253,0.420138
4,0.961300,0.918601,0.550173,0.485731,0.550173,0.507829
5,0.923900,0.922167,0.541522,0.475276,0.541522,0.497199


TrainOutput(global_step=1050, training_loss=0.9402725038074312, metrics={'train_runtime': 430.1069, 'train_samples_per_second': 39.002, 'train_steps_per_second': 2.441, 'total_flos': 4413767211110400.0, 'train_loss': 0.9402725038074312, 'epoch': 5.0})

In [ ]:
#we had our best result here at epoch 4 with lowest loss values and .507 f1

In [ ]:
roberta_model = RoBERTa()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#do it again with 4 epochs

#we evaluate training at the end of every epoch

learning_rate = 2e-5 #lower lr
batch_size = 16
num_epochs = 4 #adjust epoch

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

trainer = Trainer(
    model= roberta_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = roberta_model.get_tokenizer(),
    compute_metrics = eval_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.933981,0.525952,0.276625,0.525952,0.362561


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.933981,0.525952,0.276625,0.525952,0.362561
2,No log,0.933343,0.531142,0.477225,0.531142,0.391259
3,0.957700,0.933776,0.546713,0.479076,0.546713,0.493506
4,0.957700,0.906294,0.560554,0.488934,0.560554,0.507454


TrainOutput(global_step=840, training_loss=0.9379231407528832, metrics={'train_runtime': 345.3117, 'train_samples_per_second': 38.863, 'train_steps_per_second': 2.433, 'total_flos': 3531013768888320.0, 'train_loss': 0.9379231407528832, 'epoch': 4.0})

In [ ]:
roberta_model = RoBERTa()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#do it again with 4 epochs

learning_rate = 1e-5 #even lower lr
batch_size = 16
num_epochs = 4 # sticking with 4 epoch

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

trainer = Trainer(
    model= roberta_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = roberta_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.934516,0.525952,0.463619,0.525952,0.371542
2,No log,0.934309,0.524221,0.445280,0.524221,0.387272
3,0.962300,0.942585,0.536332,0.490795,0.536332,0.406735


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.934516,0.525952,0.463619,0.525952,0.371542
2,No log,0.934309,0.524221,0.445280,0.524221,0.387272
3,0.962300,0.942585,0.536332,0.490795,0.536332,0.406735
4,0.962300,0.925284,0.538062,0.472372,0.538062,0.491352


TrainOutput(global_step=840, training_loss=0.9490650358654203, metrics={'train_runtime': 345.1282, 'train_samples_per_second': 38.884, 'train_steps_per_second': 2.434, 'total_flos': 3531013768888320.0, 'train_loss': 0.9490650358654203, 'epoch': 4.0})

In [ ]:
#results not as good

In [ ]:
roberta_model = RoBERTa()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#do our best hyperparam with more bacth

learning_rate = 2e-5 #even lower lr
batch_size = 32 #more batch size
num_epochs = 4 # sticking with 4 epoch

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

trainer = Trainer(
    model= roberta_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = roberta_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.937307,0.524221,0.440144,0.524221,0.390070
2,No log,0.935552,0.527682,0.501425,0.527682,0.372375


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.937307,0.524221,0.440144,0.524221,0.390070
2,No log,0.935552,0.527682,0.501425,0.527682,0.372375
3,No log,0.924044,0.550173,0.507252,0.550173,0.448537
4,No log,0.920087,0.560554,0.498827,0.560554,0.502165


TrainOutput(global_step=420, training_loss=0.947864278157552, metrics={'train_runtime': 327.2426, 'train_samples_per_second': 41.009, 'train_steps_per_second': 1.283, 'total_flos': 3531013768888320.0, 'train_loss': 0.947864278157552, 'epoch': 4.0})

In [ ]:
#it was pretty good with 32 bs and 4 epochs. now im gonna try five

In [ ]:
roberta_model = RoBERTa()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#do our best hyperparam with more bacth

learning_rate = 2e-5 #even lower lr
batch_size = 32 #more batch size
num_epochs = 5 # more epoch

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

trainer = Trainer(
    model= roberta_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = roberta_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.936489,0.522491,0.431695,0.522491,0.375488


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.936489,0.522491,0.431695,0.522491,0.375488
2,No log,0.936625,0.527682,0.526289,0.527682,0.369427
3,No log,0.925976,0.529412,0.455242,0.529412,0.388052
4,No log,0.915594,0.558824,0.497532,0.558824,0.481123
5,0.942100,0.923481,0.557093,0.487537,0.557093,0.501826


TrainOutput(global_step=525, training_loss=0.9396636236281622, metrics={'train_runtime': 409.8567, 'train_samples_per_second': 40.929, 'train_steps_per_second': 1.281, 'total_flos': 4413767211110400.0, 'train_loss': 0.9396636236281622, 'epoch': 5.0})

In [ ]:
#not as good. still best is lr 2e-5, bs=16, epoch = 4
best_model_roberta = RoBERTa()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#do it again with 4 epochs

#we evaluate training at the end of every epoch

learning_rate = 2e-5 #lower lr
batch_size = 16
num_epochs = 4 #adjust epoch

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results/best-model-roberta",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

trainer = Trainer(
    model= best_model_roberta.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = best_model_roberta.get_tokenizer(),
    compute_metrics = eval_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.933981,0.525952,0.276625,0.525952,0.362561
2,No log,0.933343,0.531142,0.477225,0.531142,0.391259
3,0.957700,0.933776,0.546713,0.479076,0.546713,0.493506


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.933981,0.525952,0.276625,0.525952,0.362561
2,No log,0.933343,0.531142,0.477225,0.531142,0.391259
3,0.957700,0.933776,0.546713,0.479076,0.546713,0.493506
4,0.957700,0.906294,0.560554,0.488934,0.560554,0.507454


TrainOutput(global_step=840, training_loss=0.9379231407528832, metrics={'train_runtime': 343.1541, 'train_samples_per_second': 39.108, 'train_steps_per_second': 2.448, 'total_flos': 3531013768888320.0, 'train_loss': 0.9379231407528832, 'epoch': 4.0})

In [ ]:
test_result_random = trainer.evaluate(eval_dataset=test_random_data)

In [ ]:
print(test_result_random)

{'eval_loss': 0.9538675546646118, 'eval_accuracy': 0.5371329879101899, 'eval_precision': 0.4521382014646263, 'eval_recall': 0.5371329879101899, 'eval_f1': 0.4778357173350407, 'eval_runtime': 4.8131, 'eval_samples_per_second': 120.296, 'eval_steps_per_second': 7.687, 'epoch': 4.0}


In [ ]:
test_result_fixed = trainer.evaluate(eval_dataset=test_fixed_data)
print(test_result_fixed)

{'eval_loss': 1.0720051527023315, 'eval_accuracy': 0.4607013301088271, 'eval_precision': 0.37849501732267066, 'eval_recall': 0.4607013301088271, 'eval_f1': 0.4078549529160921, 'eval_runtime': 6.8193, 'eval_samples_per_second': 121.274, 'eval_steps_per_second': 7.625, 'epoch': 4.0}


In [ ]:
trainer.save_model('./results/best-model-roberta')

In [ ]:
trainer.save_model('./idea2-best-model')

In [ ]:
!zip -r idea2-best-model.zip ./idea2-best-model

  adding: idea2-best-model/ (stored 0%)
  adding: idea2-best-model/tokenizer_config.json (deflated 76%)
  adding: idea2-best-model/special_tokens_map.json (deflated 52%)
  adding: idea2-best-model/tokenizer.json (deflated 72%)
  adding: idea2-best-model/merges.txt (deflated 53%)
  adding: idea2-best-model/vocab.json (deflated 59%)
  adding: idea2-best-model/model.safetensors (deflated 12%)
  adding: idea2-best-model/training_args.bin (deflated 51%)
  adding: idea2-best-model/config.json (deflated 53%)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving idea2-best-model.zip to idea2-best-model.zip


In [ ]:
!unzip idea2-best-model.zip -d ./best_model

Archive:  idea2-best-model.zip
replace ./best_model/idea2-best-model/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
model_path = "./best_model/idea2-best-model/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
sentences = [
    "The phone is awesome and works as very well!",
    "This place sucks and I hope it fails.",
    "The item is okay, not great but not bad either.",
    "The delivery guy was fast, however the food is cold and the box was dented.",
    "I can't even begin to describe what I think about this place.",
    "The hotel was beautiful but the manager was rude and rolled her eyes at me.",
    "The hotel is evil, disgusting, horrible, yucky, rude, "
]

tokenized_sentences = tokenizer(sentences, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
outputs = model(**tokenized_sentences)
predictions = torch.argmax(outputs.logits, dim=1)

for sentence, prediction in zip(sentences, predictions):
    print(f"Sentence: {sentence}")
    print(f"Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): {prediction.item()}\n")

Sentence: The phone is awesome and works as very well!
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: This place sucks and I hope it fails.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: The item is okay, not great but not bad either.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: The delivery guy was fast, however the food is cold and the box was dented.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: I can't even begin to describe what I think about this place.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: The hotel was beautiful but the manager was rude and rolled her eyes at me.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: The hotel is evil, disgusting, horrible, yucky, rude, 
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

